In [11]:
#
# Created on Mon Jun 14 2021
#
# The MIT License (MIT)
# Copyright (c) 2021 Vishnu Suresh
#
# Permission is hereby granted, free of charge, to any person obtaining a copy of this software
# and associated documentation files (the "Software"), to deal in the Software without restriction,
# including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so,
# subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in all copies or substantial
# portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED
# TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT,
# TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
#

In [12]:
#Importing Needed Libaries
import nltk
import numpy as np


In [13]:
#importing data for nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Achu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
# Input String
inputStr='''
store 2 in $v1 .
add 3, 5 and $v1 .
print result.
divide 4, 2 . print result .
'''
print("\nInput String :-\n",inputStr)
# inputStr


Input String :-
 
store 2 in $v1 .
add 3, 5 and $v1 .
print result.
divide 4, 2 . print result .



In [15]:
#making Tokens of input 
tokenizeInput=np.array(nltk.word_tokenize(inputStr))
print("\nTokenzied Input :- \n",tokenizeInput,"\n")


Tokenzied Input :- 
 ['store' '2' 'in' '$' 'v1' '.' 'add' '3' ',' '5' 'and' '$' 'v1' '.'
 'print' 'result' '.' 'divide' '4' ',' '2' '.' 'print' 'result' '.'] 



In [16]:
# RightHandTree and MakeTape

class RightHandTree():
    def __init__(self):
        self.left = None
        self.right = None
        self.data = None

    def insertNode(self, value):
        print("need to add value ",value)
        if self.data == None:
            print("Value added when None ", value)
            self.data = value
        elif "$" in self.data:
            print("node contain variable need to change ",self.data)
            temp = self.data
            self.data=value
            self.right=None
            self.left = RightHandTree()
            self.left.insertNode(temp)
        elif self.right == None:
            print("Add to right value :- ",value, " Current parent :- ",self.data )
            print("Current tree :- ")
            self.PrintTree()
            self.right = RightHandTree()
            self.right.insertNode(value=value)
        else:
            self.right.insertNode(value=value)
        # else:
        #     self.right.insertNode(value=value)
        # else:
        #     if "$" in self.right.data:
        #         # print("right is not none and $ ",value)
        #         temp = self.right
        #         self.right = RightHandTree()
        #         self.right.insertNode(value=value)
        #         self.right.left = temp
        #     else:
        #         # print("right is not none ",value)
        #         self.right.insertNode(value)

    def PrintTree(self, nodePostion="root", height=0):
        print(f'{self.data} : {nodePostion}, Height : {height}'),
        if self.left:
            self.left.PrintTree(nodePostion="left", height=height+1)
        if self.right:
            self.right.PrintTree(nodePostion="right", height=height+1)

    def makeTape(self):
        tape = []
        while self.data is not None:
            tape.append(self.data)
            if self.left is not None:
                tape.append(self.left.data)
            if self.right is not None:
                self = self.right
            else:
                break
        return tape


# def makeTape(node:CustomTree):
#     tape=[]
#     while node.data is not None:
#         tape.append(node.data)
#         if node.left is not None:
#             tape.append(node.left.data)
#         if node.right is not None:
#             node=node.right
#         else:
#             break
#     return tape

In [17]:
#Word as function For Arthimetic Env
class ArithameticEnv:
    def __init__(self):
        pass
    @staticmethod
    def addFun(*args):
        result =" ".join(["result =", *[f'{x if "$" not in x else x[1:]} +' for x in args]])
        result=result[:-1]
        return result
    @staticmethod
    def multiplyFun(*args):
        result =" ".join(["result =", *[f'{x if "$" not in x else x[1:]} *' for x in args]])
        result=result[:-1]
        return result
    @staticmethod
    def divideFun(*args):
        result =" ".join(["result =", *[f'{x if "$" not in x else x[1:]} /' for x in args]])
        result=result[:-1]
        return result
    @staticmethod
    def storeFun(*args):
        if len(args)== 2:
            # if str(args[1]).isnumeric():
            result=f'{args[0] if "$" not in args[0] else args[0][1:]}={args[1] if "$" not in args[1] else args[1][1:]}'
            # else:
                # raise Exception("Second argument must be integer")
            return result
        else:
            raise Exception("Only One variable and one Integer is support")
    @staticmethod
    def toFun(*args):
        return None
    @staticmethod
    def inFun(*args):
        return None


    @staticmethod
    def andFun(*args):
        return None
    @staticmethod
    def commaFun(*args):
        ArithameticEnv.andFun(*args)
    @staticmethod
    def printfun(*args):
        result =" ".join(["print(", *[f'{x if "$" not in x else x[1:]}' for x in args],")"])
        return result
    env_Variables=["result"]
    env_Words_and_WordAsFunction={"add":addFun.__func__,"and":andFun.__func__,",":commaFun.__func__,"print":printfun.__func__,"multiply":multiplyFun.__func__,"store":storeFun.__func__,"to":toFun.__func__,"in":inFun.__func__,"divide":divideFun.__func__}
# ArithameticEnv.addFun("v1","v2")
#ArithameticEnv.addFun("$v1","v1")
# ArithameticEnv.env_Words_and_WordAsFunction['add']("$v1","v1")

In [18]:
# Making of Tape from Sentence
global_Tape=[]
itreate_each_word=iter(tokenizeInput)
catch_variable_value=False
end_Of_a_Sentence=True
node=None
try:
    while True:
        current_Word=next(itreate_each_word)
        if "$"== current_Word and not catch_variable_value:
            catch_variable_value=True
        elif "."==current_Word and not end_Of_a_Sentence:
            end_Of_a_Sentence=True
            print("\nCustom Tree Stracture of the current Sentence :- \n")
            node.PrintTree()
            tape=node.makeTape()
            global_Tape.append(tape)
            print("\nEnd of a sentence","\nTape :- \n",tape,"\n")
        else:
            if end_Of_a_Sentence:
                print("\nStarting of a sentence\n")
                end_Of_a_Sentence=False
                node=RightHandTree()
            elif catch_variable_value:
                catch_variable_value=False
                current_Word="$"+current_Word
            elif str(current_Word).isnumeric():
                current_Word="$"+current_Word
            elif current_Word in ArithameticEnv.env_Variables:
                current_Word="$"+current_Word
            node.insertNode(current_Word)
            # print(current_Word)
except StopIteration:
    print("\nEnd of All the lines\n")
    print("\nGlobal Tape :-\n",global_Tape)
    pass



Starting of a sentence

need to add value  store
Value added when None  store
need to add value  $2
Add to right value :-  $2  Current parent :-  store
Current tree :- 
store : root, Height : 0
need to add value  $2
Value added when None  $2
need to add value  in
need to add value  in
node contain variable need to change  $2
need to add value  $2
Value added when None  $2
need to add value  $v1
need to add value  $v1
Add to right value :-  $v1  Current parent :-  in
Current tree :- 
in : root, Height : 0
$2 : left, Height : 1
need to add value  $v1
Value added when None  $v1

Custom Tree Stracture of the current Sentence :- 

store : root, Height : 0
in : right, Height : 1
$2 : left, Height : 2
$v1 : right, Height : 2

End of a sentence 
Tape :- 
 ['store', 'in', '$2', '$v1'] 


Starting of a sentence

need to add value  add
Value added when None  add
need to add value  $3
Add to right value :-  $3  Current parent :-  add
Current tree :- 
add : root, Height : 0
need to add value  $3
V

In [19]:
#The machice to Transfer the natural languae to Python code
pythonCode=[]
varibleBuffer=[]
for sentenceTape in global_Tape:
    varibleBuffer=[]
    for word in reversed(sentenceTape):
        if "$" in word :
            varibleBuffer.append(word)
        else:
            result=ArithameticEnv.env_Words_and_WordAsFunction[word](*varibleBuffer)
            if result is not None:
               pythonCode.append(result)
pythonCode






['v1=2',
 'result = v1 + 5 + 3 ',
 'print( result )',
 'result = 2 / 4 ',
 'print( result )']

In [20]:
print("\npython code Exected Output\n")
exec("\n".join(pythonCode))


python code Exected Output

10
0.5
